In [9]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [10]:
df_train = pd.read_csv("act_train.csv", parse_dates=['date'])

In [11]:
df_ppl = pd.read_csv('people.csv', parse_dates=['date'])

In [12]:
df_test = pd.read_csv('act_test.csv', parse_dates=['date'])

In [62]:
df_sample_submission = pd.read_csv('sample_submission.csv')

In [14]:
df_train_merge = pd.merge(df_train, df_ppl, on='people_id')
df_test_merge = pd.merge(df_test, df_ppl, on='people_id')

In [15]:
train = pd.read_csv("train_merged_converted.csv")

In [16]:
test = pd.read_csv('test_merged_converted.csv')

In [17]:
import datetime
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import random
from operator import itemgetter
import time
import copy

random.seed(2)




In [18]:
random_state = 0

In [20]:
train.groupby('outcome').describe()

activity_category     char_10_x     char_10_y       char_11  \
outcome                                                                      
0       count       1.221794e+06  1.221794e+06  1.221794e+06  1.221794e+06   
        mean        3.000613e+00  3.629412e+02  1.384194e-01  1.246814e-01   
        std         1.275466e+00  1.161698e+03  3.453398e-01  3.303575e-01   
        min         1.000000e+00 -9.990000e+02  0.000000e+00  0.000000e+00   
        25%         2.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00   
        50%         3.000000e+00  5.000000e+00  0.000000e+00  0.000000e+00   
        75%         4.000000e+00  2.910000e+02  0.000000e+00  0.000000e+00   
        max         7.000000e+00  9.251000e+03  1.000000e+00  1.000000e+00   
1       count       9.754970e+05  9.754970e+05  9.754970e+05  9.754970e+05   
        mean        2.993449e+00  4.374892e+02  3.700801e-01  3.090271e-01   
        std         1.360862e+00  1.291335e+03  4.828261e-01  4.620926e-01   
        min         1.000000e+00 -9.990000e+02  0.000000e+00  0.000000e+00   
        25%         2.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00   
        50%         2.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00   
        75%         4.000000e+00  4.410000e+02  1.000000e+00  1.000000e+00   
        max         7.000000e+00  9.235000e+03  1.000000e+00  1.000000e+00   

                    char_12       char_13       char_14       char_15  \
outcome                                                                 
0       count  1.221794e+06  1.221794e+06  1.221794e+06  1.221794e+06   
        mean   1.496030e-01  2.122657e-01  1.586855e-01  1.582795e-01   
        std    3.566820e-01  4.089121e-01  3.653827e-01  3.650031e-01   
        min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
        25%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
        50%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
        75%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
        max    1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   
1       count  9.754970e+05  9.754970e+05  9.754970e+05  9.754970e+05   
        mean   3.415080e-01  5.213179e-01  3.759314e-01  3.918936e-01   
        std    4.742157e-01  4.995456e-01  4.843627e-01  4.881734e-01   
        min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
        25%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
        50%    0.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00   
        75%    1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   
        max    1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   

                    char_16       char_17      ...           char_8_y  \
outcome                                        ...                      
0       count  1.221794e+06  1.221794e+06      ...       1.221794e+06   
        mean   1.579530e-01  1.646489e-01      ...       3.173525e+00   
        std    3.646971e-01  3.708635e-01      ...       1.449145e+00   
        min    0.000000e+00  0.000000e+00      ...       1.000000e+00   
        25%    0.000000e+00  0.000000e+00      ...       3.000000e+00   
        50%    0.000000e+00  0.000000e+00      ...       3.000000e+00   
        75%    0.000000e+00  0.000000e+00      ...       3.000000e+00   
        max    1.000000e+00  1.000000e+00      ...       8.000000e+00   
1       count  9.754970e+05  9.754970e+05      ...       9.754970e+05   
        mean   4.064215e-01  4.238342e-01      ...       2.993560e+00   
        std    4.911653e-01  4.941650e-01      ...       1.659583e+00   
        min    0.000000e+00  0.000000e+00      ...       1.000000e+00   
        25%    0.000000e+00  0.000000e+00      ...       2.000000e+00   
        50%    0.000000e+00  0.000000e+00      ...       2.000000e+00   
        75%    1.000000e+00  1.000000e+00      ...       3.000000e+00   
        max    1.000000e+00  1.000000e+00      ...    

In [21]:
df = train.groupby('outcome').describe()

In [22]:
df.to_csv('temp.csv')

In [23]:
df = pd.read_csv('temp.csv')

In [26]:
df2 = df[df['Unnamed: 1']=='mean']

In [27]:
df2

,outcome,Unnamed: 1,activity_category,char_10_x,char_10_y,char_11,char_12,char_13,char_14,char_15,...,char_8_y,char_9_x,char_9_y,day_x,day_y,group_1,month_x,month_y,year_x,year_y
1,0,mean,3.000613,362.941195,0.138419,0.124681,0.149603,0.212266,0.158686,0.158280,...,3.173525,-922.594651,3.405861,17.129722,15.876255,18239.857416,7.142654,6.697368,2022.458395,2021.777634
9,1,mean,2.993449,437.489176,0.370080,0.309027,0.341508,0.521318,0.375931,0.391894,...,2.993560,-932.120516,3.564167,16.205061,15.749472,13658.417823,6.742545,6.894524,2022.526066,2021.818147


In [28]:
df.columns

Index(['outcome', 'Unnamed: 1', 'activity_category', 'char_10_x', 'char_10_y',
       'char_11', 'char_12', 'char_13', 'char_14', 'char_15', 'char_16',
       'char_17', 'char_18', 'char_19', 'char_1_x', 'char_1_y', 'char_20',
       'char_21', 'char_22', 'char_23', 'char_24', 'char_25', 'char_26',
       'char_27', 'char_28', 'char_29', 'char_2_x', 'char_2_y', 'char_30',
       'char_31', 'char_32', 'char_33', 'char_34', 'char_35', 'char_36',
       'char_37', 'char_38', 'char_3_x', 'char_3_y', 'char_4_x', 'char_4_y',
       'char_5_x', 'char_5_y', 'char_6_x', 'char_6_y', 'char_7_x', 'char_7_y',
       'char_8_x', 'char_8_y', 'char_9_x', 'char_9_y', 'day_x', 'day_y',
       'group_1', 'month_x', 'month_y', 'year_x', 'year_y'],
      dtype='object')

In [29]:
features = ['char_10_x', 'char_10_y',
       'char_11', 'char_12', 'char_13', 'char_14', 'char_15', 'char_16',
       'char_17', 'char_18', 'char_19', 'char_1_x', 'char_1_y', 'char_20',
       'char_21', 'char_22', 'char_23', 'char_24', 'char_25', 'char_26',
       'char_27', 'char_28', 'char_29', 'char_2_x', 'char_2_y', 'char_30',
       'char_31', 'char_32', 'char_33', 'char_34', 'char_35', 'char_36',
       'char_37', 'char_38', 'char_3_x', 'char_3_y', 'char_4_x', 'char_4_y',
       'char_5_x', 'char_5_y', 'char_6_x', 'char_6_y', 'char_7_x', 'char_7_y',
       'char_8_x', 'char_8_y', 'char_9_x', 'char_9_y', 'day_x', 'day_y',
       'group_1', 'month_x', 'month_y', 'year_x', 'year_y']

In [34]:
df2.loc[1,'char_19']

0.16017757494307552

In [43]:

for item in features:
    print(item,1/(df2.loc[1,item]/df2.loc[9,item]))
    


char_10_x 1.2053996149
char_10_y 2.67361406986
char_11 2.47853379318
char_12 2.28276210409
char_13 2.45596800719
char_14 2.36903455169
char_15 2.47595837393
char_16 2.57305404435
char_17 2.57417018861
char_18 2.54456412275
char_19 2.56115822484
char_1_x 1.01034885096
char_1_y 0.95187421556
char_20 3.00270467889
char_21 2.57183925713
char_22 2.63782312176
char_23 2.40843605057
char_24 2.63824599606
char_25 2.36913099921
char_26 2.17682468909
char_27 2.38167731805
char_28 2.54089380861
char_29 2.38197670168
char_2_x 1.01031866845
char_2_y 1.20811925273
char_30 2.33424461857
char_31 2.34773619618
char_32 2.57735920573
char_33 2.29537268598
char_34 2.41195111901
char_35 2.39672411958
char_36 2.52904163872
char_37 2.65919336731
char_38 2.74445229637
char_3_x 1.01029665018
char_3_y 0.771120203267
char_4_x 1.01029104989
char_4_y 0.838202712699
char_5_x 1.01028971122
char_5_y 0.925481681831
char_6_x 1.01028277227
char_6_y 0.781472474136
char_7_x 1.01029033043
char_7_y 0.840235773267
char_8_x 1

#  select columns which has 2.6 or bigger values

In [44]:
select_feature = ['char_10_y','char_20', 'char_22','char_24','char_37','char_38']

In [45]:
df_selected_train = train[select_feature]

In [47]:
df_selected_train.head()

,char_10_y,char_20,char_22,char_24,char_37,char_38
0,1,0,0,0,0,36
1,1,0,0,0,0,36
2,1,0,0,0,0,36
3,1,0,0,0,0,36
4,1,0,0,0,0,36


In [53]:
x_train = df_selected_train.values

In [50]:
y_train = train['outcome']

In [55]:
x_test = test[select_feature]

#  ML with Random Forest

In [52]:
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics



In [54]:
from sklearn.ensemble import RandomForestClassifier

Xtrain, Xtest, ytrain, ytest = train_test_split(x_train,y_train,random_state=0)
clf = RandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(Xtrain, ytrain)
ypred = clf.predict(Xtest)
metrics.accuracy_score(ypred, ytest)

0.84307229080158663

In [56]:
prediction = clf.predict(x_test)

In [57]:
prediction

array([1, 1, 1, ..., 0, 0, 0])

In [63]:
df_sample_submission.head()

,activity_id,outcome
0,act1_1,0
1,act1_100006,0
2,act1_100050,0
3,act1_100065,0
4,act1_100068,0


In [59]:
test.head()

,people_id,activity_id,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,...,char_32,char_33,char_34,char_35,char_36,char_37,char_38,year_y,month_y,day_y
0,ppl_100004,act1_249281,1,5,10,5,1,6,1,1,...,1,1,1,1,1,1,76,2022,7,20
1,ppl_100004,act2_230855,5,-999,-999,-999,-999,-999,-999,-999,...,1,1,1,1,1,1,76,2022,7,20
2,ppl_10001,act1_240724,1,12,1,5,4,6,1,1,...,1,1,1,1,1,1,90,2022,10,14
3,ppl_10001,act1_83552,1,20,10,5,4,6,1,1,...,1,1,1,1,1,1,90,2022,10,14
4,ppl_10001,act2_1043301,5,-999,-999,-999,-999,-999,-999,-999,...,1,1,1,1,1,1,90,2022,10,14


In [64]:
df_sample_submission['activity_id']=test['activity_id']

In [65]:
df_sample_submission.head()

,activity_id,outcome
0,act1_249281,0
1,act2_230855,0
2,act1_240724,0
3,act1_83552,0
4,act2_1043301,0


In [66]:
df_sample_submission['outcome']=prediction

In [67]:
df_sample_submission.head()

,activity_id,outcome
0,act1_249281,1
1,act2_230855,1
2,act1_240724,1
3,act1_83552,1
4,act2_1043301,1


In [68]:
df_sample_submission.to_csv('09-10-16_randomForest_1.csv', index = False)

#  ML with gradient boost

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
Xtrain, Xtest, ytrain, ytest = train_test_split(x_train,y_train, random_state=0)
gbt = GradientBoostingClassifier(max_depth=10, n_estimators=50)
gbt.fit(Xtrain, ytrain)
ypred = gbt.predict(Xtest)
metrics.accuracy_score(ypred, ytest)

#  XGBoost

In [70]:
import datetime
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import random
from operator import itemgetter
import time
import copy

random.seed(2)



In [71]:
random_state = 0

In [72]:
def get_importance(gbm, features):
    create_feature_map(features)
    importance = gbm.get_fscore(fmap='xgb.fmap')
    importance = sorted(importance.items(), key=itemgetter(1), reverse=True)
    return importance


def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()

def intersect(a, b):
    return list(set(a) & set(b))



In [73]:
eta = 0.1
max_depth = 6
subsample = 0.8
colsample_bytree = 0.8
start_time = time.time()

print('XGBoost params. ETA: {}, MAX_DEPTH: {}, SUBSAMPLE: {}, COLSAMPLE_BY_TREE: {}'.format(eta, max_depth, subsample, colsample_bytree))
params = {
        "objective": "binary:logistic",
        "booster" : "gbtree",
        "eval_metric": "auc",
        "eta": eta,
        "tree_method": 'exact',
        "max_depth": max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "silent": 1,
        "seed": 0
}

early_stopping_rounds = 10
test_size = 0.1




XGBoost params. ETA: 0.1, MAX_DEPTH: 6, SUBSAMPLE: 0.8, COLSAMPLE_BY_TREE: 0.8


In [74]:
num_boost_round = 100

In [75]:
features =  ['char_10_y','char_20', 'char_22','char_24','char_37','char_38']

In [80]:
features = ['char_10_x', 'char_10_y',
       'char_11', 'char_12', 'char_13', 'char_14', 'char_15', 'char_16',
       'char_17', 'char_18', 'char_19', 'char_1_x', 'char_1_y', 'char_20',
       'char_21', 'char_22', 'char_23', 'char_24', 'char_25', 'char_26',
       'char_27', 'char_28', 'char_29', 'char_2_x', 'char_2_y', 'char_30',
       'char_31', 'char_32', 'char_33', 'char_34', 'char_35', 'char_36',
       'char_37', 'char_38', 'char_3_x', 'char_3_y', 'char_4_x', 'char_4_y',
       'char_5_x', 'char_5_y', 'char_6_x', 'char_6_y', 'char_7_x', 'char_7_y',
       'char_8_x', 'char_8_y', 'char_9_x', 'char_9_y', 'day_x', 'day_y',
       'group_1', 'month_x', 'month_y', 'year_x', 'year_y']

In [81]:
num_boost_round = 200

In [82]:
target = 'outcome'
X_train, X_valid = train_test_split(train, test_size=test_size, random_state=random_state)
print('Length train:', len(X_train.index))
print('Length valid:', len(X_valid.index))
y_train = X_train[target]
y_valid = X_valid[target]
dtrain = xgb.DMatrix(X_train[features], y_train)
dvalid = xgb.DMatrix(X_valid[features], y_valid)

watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)

print("Validating...")
check = gbm.predict(xgb.DMatrix(X_valid[features]), ntree_limit=gbm.best_iteration+1)
score = roc_auc_score(X_valid[target].values, check)
print('Check error value: {:.6f}'.format(score))

imp = get_importance(gbm, features)
print('Importance array: ', imp)

print("Predict test set...")
test_prediction = gbm.predict(xgb.DMatrix(test[features]), ntree_limit=gbm.best_iteration+1)

print('Training time: {} minutes'.format(round((time.time() - start_time)/60, 2)))
results  = test_prediction.tolist()
print(score)

Length train: 1977561
Length valid: 219730
Will train until eval error hasn't decreased in 10 rounds.
[0]	train-auc:0.914236	eval-auc:0.914012
[1]	train-auc:0.927364	eval-auc:0.927040
[2]	train-auc:0.930530	eval-auc:0.930100
[3]	train-auc:0.931309	eval-auc:0.930681
[4]	train-auc:0.932235	eval-auc:0.931749
[5]	train-auc:0.932460	eval-auc:0.932110
[6]	train-auc:0.932487	eval-auc:0.932251
[7]	train-auc:0.933023	eval-auc:0.932803
[8]	train-auc:0.934069	eval-auc:0.933789
[9]	train-auc:0.934331	eval-auc:0.934048
[10]	train-auc:0.934775	eval-auc:0.934528
[11]	train-auc:0.935324	eval-auc:0.935095
[12]	train-auc:0.936180	eval-auc:0.935939
[13]	train-auc:0.936846	eval-auc:0.936623
[14]	train-auc:0.936939	eval-auc:0.936750
[15]	train-auc:0.937149	eval-auc:0.936916
[16]	train-auc:0.937302	eval-auc:0.937039
[17]	train-auc:0.938080	eval-auc:0.937837
[18]	train-auc:0.938563	eval-auc:0.938319
[19]	train-auc:0.939736	eval-auc:0.939436
[20]	train-auc:0.939860	eval-auc:0.939586
[21]	train-auc:0.940015	ev

In [83]:
def create_submission(score, test, prediction):
    now = datetime.datetime.now()
    sub_file = 'submission_' + str(score) + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    print('Writing submission: ', sub_file)
    f = open(sub_file, 'w')
    f.write('activity_id,outcome\n')
    total = 0
    for id in test['activity_id']:
        str1 = str(id) + ',' + str(prediction[total])
        str1 += '\n'
        total += 1
        f.write(str1)
    f.close()




In [84]:
create_submission(score, test, test_prediction)

Writing submission:  submission_0.97105526533_2016-09-10-11-49.csv
